In [72]:
from sklearn.naive_bayes import CategoricalNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

# from datetime import datetime

# matplotlib inline

In [73]:
df = pd.read_csv('recent-grads-new-features.csv')
df.head()

,Unnamed: 0,Rank,Major_code,Major,Major_category,Total,Sample_size,Men,Women,ShareWomen,...,Major_category_Education,Major_category_Engineering,Major_category_Health,Major_category_Humanities & Liberal Arts,Major_category_Industrial Arts & Consumer Services,Major_category_Interdisciplinary,Major_category_Law & Public Policy,Major_category_Physical Sciences,Major_category_Psychology & Social Work,Major_category_Social Science
0,0,1,2419,PETROLEUM ENGINEERING,Engineering,2339,36,2057,282,0.120564,...,0,1,0,0,0,0,0,0,0,0
1,1,2,2416,MINING AND MINERAL ENGINEERING,Engineering,756,7,679,77,0.101852,...,0,1,0,0,0,0,0,0,0,0
2,2,3,2415,METALLURGICAL ENGINEERING,Engineering,856,3,725,131,0.153037,...,0,1,0,0,0,0,0,0,0,0
3,3,4,2417,NAVAL ARCHITECTURE AND MARINE ENGINEERING,Engineering,1258,16,1123,135,0.107313,...,0,1,0,0,0,0,0,0,0,0
4,4,5,2405,CHEMICAL ENGINEERING,Engineering,32260,289,21239,11021,0.341631,...,0,1,0,0,0,0,0,0,0,0


In [76]:
features = df[["Major_category", "Num_majors", "Women", "ShareWomen", "Percent_college", "Percent_non_college", "Percent_low_wage", "Percent_sample_size", "Unemployment_rate", "P75th", "P25th", "Part_time"]]

medians = df[["Median_category"]]

In [77]:
features.head()

,Major_category,Num_majors,Women,ShareWomen,Percent_college,Percent_non_college,Percent_low_wage,Percent_sample_size,Unemployment_rate,P75th,P25th,Part_time
0,Engineering,29.0,282,0.120564,0.655836,0.155622,0.082514,0.015391,0.018381,125000,95000,270
1,Engineering,29.0,77,0.101852,0.462963,0.339947,0.066138,0.009259,0.117241,90000,55000,170
2,Engineering,29.0,131,0.153037,0.532710,0.205607,0.000000,0.003505,0.024096,105000,50000,133
3,Engineering,29.0,135,0.107313,0.420509,0.081081,0.000000,0.012719,0.050125,80000,43000,150
4,Engineering,29.0,11021,0.341631,0.567700,0.137632,0.030130,0.008958,0.061098,75000,50000,5180


In [78]:
le = LabelEncoder()
le.fit(df["Major_category"])
# le.transform(features["Major_category"])

LabelEncoder()

In [79]:
features["Major_category"] = le.transform(features["Major_category"])

/srv/jupyter_python3-extras/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [80]:
features.head()

,Major_category,Num_majors,Women,ShareWomen,Percent_college,Percent_non_college,Percent_low_wage,Percent_sample_size,Unemployment_rate,P75th,P25th,Part_time
0,7,29.0,282,0.120564,0.655836,0.155622,0.082514,0.015391,0.018381,125000,95000,270
1,7,29.0,77,0.101852,0.462963,0.339947,0.066138,0.009259,0.117241,90000,55000,170
2,7,29.0,131,0.153037,0.532710,0.205607,0.000000,0.003505,0.024096,105000,50000,133
3,7,29.0,135,0.107313,0.420509,0.081081,0.000000,0.012719,0.050125,80000,43000,150
4,7,29.0,11021,0.341631,0.567700,0.137632,0.030130,0.008958,0.061098,75000,50000,5180


In [81]:
X_train, X_test, y_train, y_test = train_test_split(features, medians, test_size=0.2, random_state=42)

In [82]:
naive_bayes = CategoricalNB()
naive_bayes.fit(X_train, y_train)

/srv/jupyter_python3-extras/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


CategoricalNB()

In [83]:
predictions = naive_bayes.predict(X_test)
naive_bayes.score(X_test, y_test)

Accuracy Score 0.6


0.11428571428571428